In [80]:
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup

# Get all links from url
def get_company_links(url = "https://www.thomasnet.com/nsearch.html?act=C&cov=NA&heading=30772305&navsec=modify&what=Food+Products&which=prod"):
  response = requests.get(url)
  html = response.content
  soup = BeautifulSoup(html, "html.parser")

  company_links = []
  for title in soup.find_all("h2", class_="profile-card__title"):
    company_links.append(title.find_all("a")[0]["href"])

  return company_links

# Get info from specific page
def get_info_from_link(link):
    company_url = "https://www.thomasnet.com" + link
    company_response = requests.get(company_url)
    company_html = company_response.content
    company_soup = BeautifulSoup(company_html, "html.parser")

    name = company_soup.find("h1", class_="copro-supplier-name").text.replace("\n", "")
    address = company_soup.find("span", class_="copro-address-line").text.replace("\n", "")
    website = company_soup.find("a", class_="text")['href'].replace("\n", "")

    business_description = company_soup.find("div", id="copro_pdm")
    if business_description is not None:
      business_description = business_description.find("p").text
     
    #This element is not getting all values ​​at least XPATH
    phone_number = etree.HTML(str(company_soup)).xpath('//*[@id="copro_naft"]/div[1]/div/p[2]/span[2]')
    if phone_number:
      phone_number = phone_number[0].text
    else:
      phone_number = None

    # Extract relevant information from the page
    return {'name':name, 
                 'address':address,
                 'website':website,
                 'business_description':business_description,
                  'phone_number': phone_number}
    
#Consolidating data and return a dataframe
def consolidating_data(company_links = get_company_links()):
  data = []

  for link in company_links:
    data.append(get_info_from_link(link))

  return pd.DataFrame(data)

In [81]:
df = consolidating_data()
df

,name,address,website,business_description,phone_number
0,ETA Lumis Foods,"Bridgeton, NJ 08302",http://www.etalumisfoods.com,Custom manufacturer of extruded plant-based fo...,None
1,Superior Supplement Manufacturing,"Fountain Valley, CA 92708",https://www.superiorsupplementmfg.com,Manufacturer of standard and custom private la...,None
2,Tru Body Wellness,"Ottawa, IL 61350",https://www.trubodywellness.com/,Private label manufacturer and custom formulat...,None
3,"SupplyOne, Inc.","Newtown Square, PA 19073 | Locations",https://www.supplyone.com/,Manufacturer of standard and custom kitchen an...,844-287-4028
4,"Sonneborn, LLC","Petrolia, PA 16050",http://www.sonneborn.com,Custom manufacturer of hydrocarbon products fo...,None
5,Nutra Coast Supplements,"Leesburg, FL 34748",https://nutracoastsupplements.com/,Custom manufacturer of nutritional powders. Pa...,None
6,The Flavor Factory,"Norco, CA 92860",https://www.flavorfactory.net/,Custom manufacturer of food ingredients includ...,None
7,"Bactolac Pharmaceutical, Inc.","Hauppauge, NY 11788",https://bactolac.com/,Private label contract manufacturer of organic...,None
8,Clint's Brands,"San Antonio, TX 78209",https://www.clintsprivatepremiumbrands.com/,Manufacturer of private label food products in...,None
9,Mike Shea's Coffee Roasting Co.,"Bridgewater, MA 02324",https://mikesheascoffee.com/,Manufacturer of food products including muffin...,None


In [79]:
df.to_excel('output.xlsx', sheet_name='Sheet1')